Model simulation Strain_4 (Knock-out sucA). By knockint-out sucA we delete another reaction that consume 2-oxoglutarate.

In [48]:
from cobra.io import read_sbml_model
model = read_sbml_model('modified_model.xml')
model

Name,iJO1366
Memory address,27cc54d8c10
Number of metabolites,1810
Number of reactions,2592
Number of genes,1368
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


Based on the paper another knock-out that will increase 2-oxoglutarate concentration is sucA 


In [49]:
# research of sucA gene in our model
sucA = None
for gene in model.genes:
    if "sucA" in gene.name:
        sucA = gene.id
        break

print("Gene ID for sucA in model:", sucA)

Gene ID for sucA in model: b0726


In [50]:
model.genes.b0726 

Gene identifier,b0726
Name,sucA
Memory address,0x27cc4877f10
Functional,True
In 1 reaction(s),AKGDH


Let's search for the connected reaction to konck it out

In [51]:
model.reactions.AKGDH


Reaction identifier,AKGDH
Name,2-Oxogluterate dehydrogenase
Memory address,0x27cc63fb7d0
Stoichiometry,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c 2-Oxoglutarate + Coenzyme A + Nicotinamide adenine dinucleotide --> CO2 CO2 + Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA
GPR,b0116 and b0726 and b0727
Lower bound,0.0
Upper bound,1000.0


Test the production of EFE reaction before the knock-out

In [52]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9823718127269851 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.780573951434883 mmol/gDW*h


Knock-out of the reaction connected to sucA gene

In [53]:
model.reactions.AKGDH.knock_out()
model.reactions.AKGDH


Reaction identifier,AKGDH
Name,2-Oxogluterate dehydrogenase
Memory address,0x27cc63fb7d0
Stoichiometry,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c 2-Oxoglutarate + Coenzyme A + Nicotinamide adenine dinucleotide --> CO2 CO2 + Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA
GPR,b0116 and b0726 and b0727
Lower bound,0
Upper bound,0


Maximum theoretical biomass and EFE productivity after the knock-out

In [54]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9777223396312746 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.780573951434809 mmol/gDW*h


Combination of the new strain with media implementation

In [55]:
YEPDmedium = model.medium
YEPDmedium['EX_glc__D_e'] = 40
YEPDmedium['EX_sucr_e'] = 0
YEPDmedium['EX_ala__L_e'] = 20
YEPDmedium['EX_arg__L_e'] = 20
YEPDmedium['EX_asn__L_e'] = 20
YEPDmedium['EX_cys__L_e'] = 20
YEPDmedium['EX_gln__L_e'] = 20
YEPDmedium['EX_glu__L_e'] = 20
YEPDmedium['EX_gly_e'] = 0
YEPDmedium['EX_his__L_e'] = 0
YEPDmedium['EX_ile__L_e'] = 0
YEPDmedium['EX_leu__L_e'] = 0
YEPDmedium['EX_met__L_e'] = 0
YEPDmedium['EX_phe__L_e'] = 0
YEPDmedium['EX_pro__L_e'] = 20
YEPDmedium['EX_ser__L_e'] = 20
YEPDmedium['EX_thr__L_e'] = 20
YEPDmedium['EX_trp__L_e'] = 20
YEPDmedium['EX_tyr__L_e'] = 0
YEPDmedium['EX_val__L_e'] = 0
YEPDmedium['EX_lys__L_e'] = 0
YEPDmedium['EX_asp__L_e'] = 20

In [56]:
with model:
    model.medium = YEPDmedium
    solution = model.optimize()
    print('Statistically best (With Sucrose instead of Glucose):')
    print('Maximum theoretical growth-rate:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')
    

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Statistically best (With Sucrose instead of Glucose):
Maximum theoretical growth-rate: 16.071879446682594 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 225.35283018867955 mmol/gDW*h


Maximum theoretical EFE productivity and yield after the knock-out without the media implementation

In [57]:
model.objective = model.reactions.eth_tp
eth_production = model.optimize().objective_value
print('Maximum productivity =', eth_production, 'mmol/gDW*h')
   
eth_maximum_yield = eth_production / (-1*(model.reactions.EX_glc__D_e.flux))
print('Maximum theoretical yield =', eth_maximum_yield, 'mmol-eth/mmol-glc')

Maximum productivity = 12.780573951434837 mmol/gDW*h
Maximum theoretical yield = 1.2780573951434837 mmol-eth/mmol-glc
